In [ ]:
from tqdm import tqdm

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

In [ ]:
df = pd.read_csv('Advanced_feat_data.csv')

In [ ]:
df = df.dropna()

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,...,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,first_word_eq,last_word_eq,abs_len_diff,mean_len
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,...,0.833333,0.714286,1.0,0.857143,0.916667,0.785714,1.0,0.0,2.0,13.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,...,0.500000,0.222222,0.5,0.500000,0.500000,0.307692,1.0,0.0,5.0,10.5
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,...,0.400000,0.333333,0.4,0.250000,0.400000,0.285714,1.0,0.0,4.0,12.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,2.0,10.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,...,0.200000,0.100000,1.0,0.666667,0.428571,0.230769,1.0,0.0,6.0,10.0


In [ ]:
df_tf = df[['id', 'question1', 'question2']][0:]

In [ ]:
df_tf.head()

,id,question1,question2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...
4,4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

questions = list(df_tf['question1']) + list(df_tf['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md
import numpy as np
import pandas as pd
from tqdm import tqdm
import spacy

# Load the medium-sized English language model
nlp = spacy.load('en_core_web_md')

vecs1 = []

# tqdm is used to print progress bar in result
for qu1 in tqdm(list(df_tf['question1'])):
    doc1 = nlp(qu1)
    # 300 is the no. of dimensions of vectors for 'en_core_web_md'
    mean_vec1 = np.zeros(300)
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    vecs1.append(mean_vec1)
df_tf['q1_feats_m'] = list(vecs1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 402572/402572 [1:08:36<00:00, 97.78it/s] 


In [ ]:
vecs2 = []

# tqdm is used to print progress bar in result
for qu2 in tqdm(list(df_tf['question2'])):
    doc2 = nlp(qu2)
    # 300 is the no. of dimensions of vectors for 'en_core_web_md'
    mean_vec2 = np.zeros(300)
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    vecs2.append(mean_vec2)
df_tf['q2_feats_m'] = list(vecs2)

100%|██████████| 402572/402572 [1:06:40<00:00, 100.62it/s]


In [ ]:
df3= df_tf.drop(['id','question1','question2'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [ ]:
df_tf.head()

,id,question1,question2,q1_feats_m,q2_feats_m
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,"[-15.706554532051086, 73.145712018013, -261.50...","[-5.153241157531738, 79.66935384273529, -249.7..."
1,1,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,"[-48.87001037597656, 8.45869493484497, -25.958...","[-106.8676450252533, 47.23387098312378, -127.8..."
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,"[-44.23683786392212, 120.54541063308716, -164....","[-87.89710783958435, 85.43202686309814, -119.1..."
3,3,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,"[144.84441757202148, 99.2542234659195, -117.73...","[-203.51137495040894, -35.45449995994568, -36...."
4,4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,"[14.860610365867615, -74.24747252464294, -174....","[-11.601820707321167, 18.677152395248413, -59...."


In [ ]:
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-15.706555,73.145712,-261.501331,-23.247625,153.163697,49.914058,-70.594154,166.147006,-254.128580,1.911570,...,113.781129,-125.356327,-110.585957,83.953331,2.089845,17.695681,-18.872447,-159.929795,-135.752519,85.471042
1,-48.870010,8.458695,-25.958556,-61.946737,111.484478,-48.167946,34.420124,156.464216,-4.353240,-17.501519,...,39.277746,44.454936,66.967229,46.795848,-38.260121,-8.453460,8.856954,-31.051793,-51.180928,3.059027
2,-44.236838,120.545411,-164.491086,53.805420,197.394464,-28.971412,72.898095,376.550182,-189.784926,69.973760,...,-46.157192,-37.611517,83.218392,34.732064,-57.563090,154.393877,119.640444,-214.393299,-202.783340,64.671596
3,144.844418,99.254223,-117.736340,-201.832321,-99.472701,59.331452,16.477002,229.072741,-152.984258,57.997643,...,141.807342,-162.086170,73.374706,-79.242667,-14.113889,-99.674997,72.029673,93.612271,-187.336368,232.224180
4,14.860610,-74.247473,-174.233154,68.841327,185.905591,-218.528425,0.364984,307.240423,102.713599,-65.058079,...,65.041927,-135.954536,214.644799,-61.390936,-206.234488,101.007052,286.996062,-34.212672,37.187245,-93.786933


In [ ]:
All_featurizations = pd.concat([df,df3_q1,df3_q2],axis=1)

In [ ]:
All_featurizations.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,...,290,291,292,293,294,295,296,297,298,299
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,...,90.842673,-109.121875,-103.461947,93.912287,24.555147,-13.950214,-14.532000,-128.323186,-125.105671,87.540304
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,...,60.318785,36.759359,89.701675,68.875638,-87.536221,31.325016,-88.937340,66.108022,-117.743915,61.278787
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,...,73.656094,-4.256963,172.464854,1.855481,-85.113197,64.203202,146.907226,-253.900569,-170.002529,168.029946
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,...,77.860768,-71.490774,-25.844452,-7.730651,-86.217268,67.093756,-45.410198,-158.506922,-10.452024,61.258758
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,...,109.881261,-19.280973,109.475905,1.795609,-88.208753,33.786041,58.574612,-33.243239,-53.404904,3.535569


In [ ]:
All_featurizations.to_csv('ML_modelling_data.csv')

In [ ]:
All_featurizations.shape

(402572, 634)

In [ ]:
All_featurizations.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
...,...
295,0
296,0
297,0
298,0
